***Import packages, some may not be useful***

In [1]:
import os
import random
import nltk
from scipy import stats

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import re
import string
%matplotlib inline
plt.style.use('ggplot')

In [3]:
import operator
import math
from functools import partial

***Set up the stopping words***

In [ ]:
stop_words = set(stopwords.words("english"))

***Load the data, you should change the address where you store the data***

In [ ]:
sepsis = pd.read_csv('/Users/zhanghengqian/Desktop/DIHI/sepsis_labs_v2.csv',encoding = "ISO-8859-1")

***Simple data preprocessing***

In [ ]:
##change unit to lower case
sepsis['REFERENCE_UNIT']      = sepsis['REFERENCE_UNIT'].str.lower()
##convert string into time stamp format
sepsis['PCR_RESULT_DATE']     = pd.to_datetime(sepsis['PCR_RESULT_DATE'])
sepsis['ORDER_TIME']          = pd.to_datetime(sepsis['ORDER_TIME'])
sepsis['RESULT_TIME']         = pd.to_datetime(sepsis['RESULT_TIME'])
sepsis['COLLECTED_DATE_TIME'] = pd.to_datetime(sepsis['COLLECTED_DATE_TIME'])
sepsis['RECEIVED_DATE_TIME']  = pd.to_datetime(sepsis['RECEIVED_DATE_TIME'])
##create time difference: result time -  order time
sepsis['TIME_DIFF']           = (sepsis['RESULT_TIME'] - sepsis['ORDER_TIME']).astype('timedelta64[m]')

***Dealing With Component Name in Sepsis Data Set I***

In [ ]:
##remove all the non beaker team data and missing collected and received time data
##after that, component only wiht BKR and BKRREF
sepsis_bkr_nm = sepsis.loc[lambda df: (df.COMPONENT_NAME.str.contains('BKR')) & (df.COLLECTED_DATE_TIME.isnull() == False) & (df.RECEIVED_DATE_TIME.isnull() == False)]

In [ ]:
##remove BKR AND BKRREF from component names
##you will see the warnings: A value is trying to be set on a copy of a slice from a DataFrame. Just ignore the warnings
sepsis_bkr_nm['COMPONENT_NAME'] = sepsis_bkr_nm.COMPONENT_NAME.str.replace(r"(\s+\(BKR\))|(\s+\(BKRREF\))","")

***Standardize the component names***

In [ ]:
s_33 = sepsis_bkr_nm.COMPONENT_NAME.value_counts()

In [ ]:
#Process function
def standardizer(strings):
    filtered_sentence = []
    for w in nltk.word_tokenize(strings):
        if w not in stop_words | set(string.punctuation):
             filtered_sentence.append(w.lower())
    return filtered_sentence

In [ ]:
#Get Standard component name for each unique component name
s_33_clean = list(map(standardizer,s_33.index.values))

In [ ]:
#Create dictionary: COMPONENT_NAME, STANDARAD COMPONENT NAME
s_33_total = dict(zip(s_33.index.values, s_33_clean))

In [ ]:
##Create new column standardized component
##you will see the warnings: A value is trying to be set on a copy of a slice from a DataFrame. Just ignore the warnings
sepsis_bkr_nm['STD_COMPONENT'] = sepsis_bkr_nm['COMPONENT_NAME'].map(s_33_total)

***Dealing With Component Name in Sepsis Data Set II***

In [ ]:
##new columns time difference: colleted time -  order time
##you will see the warnings: A value is trying to be set on a copy of a slice from a DataFrame. Just ignore the warnings
sepsis_bkr_nm['TIME_DIFF_otc'] = (sepsis_bkr_nm['COLLECTED_DATE_TIME'] - sepsis_bkr_nm['ORDER_TIME']).astype('timedelta64[s]')

In [ ]:
##new columns time difference: recieved time - collected time
##you will see the warnings: A value is trying to be set on a copy of a slice from a DataFrame. Just ignore the warnings
sepsis_bkr_nm['TIME_DIFF_ctr'] = (sepsis_bkr_nm['RECEIVED_DATE_TIME'] - sepsis_bkr_nm['COLLECTED_DATE_TIME']).astype('timedelta64[s]')

In [ ]:
##new columns time difference: result time - recieved time
##you will see the warnings: A value is trying to be set on a copy of a slice from a DataFrame. Just ignore the warnings
sepsis_bkr_nm['TIME_DIFF_rtr'] = (sepsis_bkr_nm['RESULT_TIME'] - sepsis_bkr_nm['RECEIVED_DATE_TIME']).astype('timedelta64[s]')

In [ ]:
##create new columns which extract the hour from ORDER_TIME, COLLECTED_TIME, RECEIVED_TIME
##you will see the warnings: A value is trying to be set on a copy of a slice from a DataFrame. Just ignore the warnings
sepsis_bkr_nm['order_hour'] = sepsis_bkr_nm.ORDER_TIME.dt.hour
sepsis_bkr_nm['coll_hour'] = sepsis_bkr_nm.COLLECTED_DATE_TIME.dt.hour
sepsis_bkr_nm['rece_hour'] = sepsis_bkr_nm.RECEIVED_DATE_TIME.dt.hour

***Score computing functions***

In [ ]:
##these functions compute all things about the result
def sepsis_median(name):
    a = sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name) & (df.ORD_NUM_VALUE != 9999999)].ORD_NUM_VALUE.median()
    return a

def sepsis_mean(name):
    a = sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name) & (df.ORD_NUM_VALUE != 9999999)].ORD_NUM_VALUE.mean()
    return a

def sepsis_max(name):
    a = sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name) & (df.ORD_NUM_VALUE != 9999999)].ORD_NUM_VALUE.max()
    return a

def sepsis_min(name):
    a = sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name) & (df.ORD_NUM_VALUE != 9999999)].ORD_NUM_VALUE.min()
    return a

##these functions compute all things about the time difference between result time and received time
def sepsis_mean_time(name):
    a = sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name) & (df.TIME_DIFF_rtr < 86400)].TIME_DIFF_rtr.mean()
    return a

def sepsis_median_time(name):
    a = sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name) & (df.TIME_DIFF_rtr < 86400)].TIME_DIFF_rtr.median()
    return a

def sepsis_max_time(name):
    a = sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name) & (df.TIME_DIFF_rtr < 86400)].TIME_DIFF_rtr.max()
    return a

def sepsis_min_time(name):
    a = sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name) & (df.TIME_DIFF_rtr < 86400)].TIME_DIFF_rtr.min()
    return a

##distribution comparison function
##try not to use it at first, use the histogram instead
def ks_bun(name1,name2):  
    a = stats.ks_2samp(sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name1) & (df.ORD_NUM_VALUE != 9999999)].ORD_NUM_VALUE, sepsis_bkr_nm.loc[lambda df: (df.COMPONENT_NAME == name2) & (df.ORD_NUM_VALUE != 9999999)].ORD_NUM_VALUE)
    return a

***Dealing with two panels' components: CBC and CMP***

In [ ]:
sepsis_cmp = sepsis_bkr_nm.loc[lambda df: df.ORDER_DESC == 'COMPREHENSIVE METABOLIC PANEL (CMP)'].COMPONENT_NAME.value_counts()

In [ ]:
sepsis_cbc = sepsis_bkr_nm.loc[lambda df: df.ORDER_DESC == 'COMPLETE BLOOD COUNT (CBC)'].COMPONENT_NAME.value_counts()

In [ ]:
##Combined two panels' component names
sepsis_ini = np.append(sepsis_cmp.index.values,sepsis_cbc.index.values)

In [ ]:
##unify the units: x10^9/l,x109/l
regex_pat = re.compile(r' |\^|\x88')
sepsis_bkr_nm.REFERENCE_UNIT = sepsis_bkr_nm.REFERENCE_UNIT.str.replace(regex_pat, '')

In [ ]:
ref_unit = sepsis_bkr_nm.loc[lambda df: df.COMPONENT_NAME.isin(sepsis_ini)][['COMPONENT_NAME','REFERENCE_UNIT']].drop_duplicates(['COMPONENT_NAME','REFERENCE_UNIT']).dropna()
ref_unit.index = range(ref_unit.shape[0])
##Components: ALBUMIN,TOTAL PROTEIN,HEMATOCRIT have more one units.
##There is only one dorminant units, all other units are removed
##I hard code the removing step below. It works fine if you follow all my step
##Try to check ref_unit after you run this cell to see if there still exist duplicates for some component
ref_unit.drop(ref_unit.index[[29,31,32]], inplace = True)
ref_unit.index = range(ref_unit.shape[0])

***Remove all missing values and abnormal non-missing values defined in second presentation***

In [ ]:
sepsis_bkr_nm = sepsis_bkr_nm.loc[lambda df: ~df.ORD_VALUE.isnull()]

In [ ]:
sepsis_bkr_nm = sepsis_bkr_nm.loc[lambda df: df.ORD_NUM_VALUE != 9999999.0]

In [ ]:
sepsis_bkr_nm = sepsis_bkr_nm.loc[lambda df: ~df.REFERENCE_UNIT.isnull()]

***Build two panels' component table***

In [ ]:
##This table contain three features Component name, Reference Unit and Standard component name
ref_dict = sepsis_bkr_nm.loc[lambda df: df.COMPONENT_NAME.isin(ref_unit.COMPONENT_NAME.values)][['COMPONENT_NAME','REFERENCE_UNIT','STD_COMPONENT']].drop_duplicates(['COMPONENT_NAME','REFERENCE_UNIT']).dropna()
ref_dict.index = range(ref_dict.shape[0])
ref_dict.drop(ref_dict.index[[29,31,32]],inplace = True)
ref_dict.index = range(ref_dict.shape[0])

In [ ]:
##Take a first look at this table
ref_dict.head()

***Deal with the left top 50 componets***

In [ ]:
top_100 = sepsis_bkr_nm.COMPONENT_NAME.value_counts().iloc[0:100]

In [ ]:
top100_left20 = list(top100_left.iloc[0:20].index.values)

In [ ]:
ref_dict20 = sepsis_bkr_nm.loc[lambda df: df.COMPONENT_NAME.isin(top100_left.iloc[0:20].index.values)][['COMPONENT_NAME','REFERENCE_UNIT','STD_COMPONENT']].drop_duplicates(['COMPONENT_NAME','REFERENCE_UNIT']).dropna()

***First Match standardized component name and unit***

In [ ]:
##This step will append all non matached term into ref_dict table
n = len(top100_left20)
top100_left20_copy = list(top100_left20)
for i in range(n):
    aa = lambda a: any(x in s_33_total[top100_left20[i]] for x in a)
    hehe = ref_dict20.loc[lambda df: df.COMPONENT_NAME == top100_left20[i]].REFERENCE_UNIT.values[0]
    tr = ref_dict.loc[list(map(aa, ref_dict.STD_COMPONENT.values))].loc[lambda df: df.REFERENCE_UNIT == hehe].shape[0]
    
    if(tr == 0):
        ref_dict = ref_dict.append(ref_dict20.loc[lambda df: df.COMPONENT_NAME == top100_left20[i]],ignore_index=True)
        top100_left20_copy.remove(top100_left20[i])

***Show all the left component names***

In [ ]:
top100_left20_copy

***Score based approach example: POC GLUCOSE WB***

In [ ]:
##Show candidate component names
aa = lambda a: any(x in s_33_total['POC GLUCOSE WB'] for x in a)
hehe = ref_dict20.loc[lambda df: df.COMPONENT_NAME == 'POC GLUCOSE WB'].REFERENCE_UNIT.values[0]
ref_dict.loc[list(map(aa, ref_dict.STD_COMPONENT.values))].loc[lambda df: df.REFERENCE_UNIT == hehe]

In [ ]:
##Compute and show median result both target and candidate component name
sm1 = sepsis_median('GLUCOSE'), sm2 = sepsis_median('POC GLUCOSE WB')
sm1,sm2

In [ ]:
##Compute and show median time difference both target and candidate component name(minute level)
smt1 = sepsis_median_time('GLUCOSE')/60,smt2 = sepsis_median_time('POC GLUCOSE WB')/60
smt1,smt2

In [ ]:
##Compute result score
r = abs(sm1-sm2)/np.minimum(sm1,sm2)
r

In [ ]:
##Compute time difference score
t = abs(smt1-smt2)/np.minimum(smt1,smt2)
t

In [ ]:
##I give result weight 2 and give time difference weight 1
(r*2+t)/3

In [ ]:
##Based on the score above and your judgement to decide if you need to run this code
ref_dict = ref_dict.append(ref_dict20.loc[lambda df: df.COMPONENT_NAME == 'POC GLUCOSE WB'],ignore_index=True)

***All the other component can follow the same procedure***